In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing, model_selection, metrics, ensemble
import pyarrow.parquet as pq
import pickle
import itertools
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.ensemble import BalancedRandomForestClassifier as RandomForestClassifier
from scipy.stats import randint as sp_randint
from sklearn.utils import class_weight
import os

import plotly.graph_objects as go
from astropy.time import Time
from time import time
import json
from json import JSONEncoder

Lectura de archivo df_paula

In [10]:
path = './datos_exp4/df_paula.csv'
df_paula = pd.read_csv(path)
df_paula = df_paula.drop(['Unnamed: 0'],axis=1)
print(f'total eventos: {len(df_paula)}')
df_paula.head()

total eventos: 6920


,oid,Amplitude_1,Amplitude_2,AndersonDarling_1,AndersonDarling_2,Autocor_length_1,Autocor_length_2,Beyond1Std_1,Beyond1Std_2,Con_1,...,rb,sgscore1,W1-W2,W2-W3,r-W3,r-W2,g-W3,g-W2,delta_period_1,delta_period_2
0,ZTF17aaaemke,0.319945,0.266913,0.999845,0.999997,1.0,1.0,0.361111,0.375000,0.0,...,0.885476,0.998750,-0.025,1.044,3.225166,2.181166,4.193614,3.149614,1.885743e-06,4.405883e-08
1,ZTF17aaafyya,0.376849,0.347666,1.000000,1.000000,1.0,1.0,0.421875,0.388889,0.0,...,0.781429,0.996250,-0.046,0.244,2.550194,2.306194,3.454954,3.210954,2.254299e-06,3.267149e-08
2,ZTF17aaageae,0.218249,0.327319,1.000000,0.999997,1.0,1.0,0.400000,0.115385,0.0,...,0.760000,1.000000,0.023,0.629,1.937573,1.308573,2.133469,1.504469,6.688907e-05,6.721974e-05
3,ZTF17aaaivsr,0.344626,0.305270,0.999993,0.821926,1.0,1.0,0.384615,0.400000,0.0,...,0.780000,0.992012,-0.060,1.885,4.006276,2.121276,4.565984,2.680984,1.782890e-06,1.782890e-06
4,ZTF17aaaizej,0.660175,0.404425,1.000000,1.000000,1.0,1.0,0.317073,0.311111,0.0,...,0.840238,1.000000,0.009,2.871,4.185692,1.314692,4.359793,1.488793,4.163987e-08,6.982180e-05


Lectura de archivos con las caracteristicas + tensor

In [11]:
folder = './datos_exp4/'
features_4x4x10 = 'features_4x4x10.csv'
features_5x4x5 = 'features_5x4x5.csv'
features_2x4x5 = 'features_2x4x5.csv'

path = folder + features_4x4x10

df_features_tensor = pd.read_csv(path)
df_features_tensor = df_features_tensor.drop(['Unnamed: 0'],axis=1)
print(f'total eventos: {len(df_features_tensor)}')
df_features_tensor.head()

total eventos: 6909


,oid,Amplitude_1,Amplitude_2,AndersonDarling_1,AndersonDarling_2,Autocor_length_1,Autocor_length_2,Beyond1Std_1,Beyond1Std_2,Con_1,...,sgscore1,W1-W2,W2-W3,r-W3,r-W2,g-W3,g-W2,delta_period_1,delta_period_2,Tensor
0,ZTF17aaaemke,0.319945,0.266913,0.999845,0.999997,1.0,1.0,0.361111,0.375000,0.0,...,0.998750,-0.025,1.044,3.225166,2.181166,4.193614,3.149614,1.885743e-06,4.405883e-08,[ 1.65146666e+01 -1.00000000e+03 1.64708068e+...
1,ZTF17aaafyya,0.376849,0.347666,1.000000,1.000000,1.0,1.0,0.421875,0.388889,0.0,...,0.996250,-0.046,0.244,2.550194,2.306194,3.454954,3.210954,2.254299e-06,3.267149e-08,[1.54776381e+01 1.55176913e+01 1.56471603e+01 ...
2,ZTF17aaageae,0.218249,0.327319,1.000000,0.999997,1.0,1.0,0.400000,0.115385,0.0,...,1.000000,0.023,0.629,1.937573,1.308573,2.133469,1.504469,6.688907e-05,6.721974e-05,[14.6432357 14.40605199 14.5821878 14.627640...
3,ZTF17aaaivsr,0.344626,0.305270,0.999993,0.821926,1.0,1.0,0.384615,0.400000,0.0,...,0.992012,-0.060,1.885,4.006276,2.121276,4.565984,2.680984,1.782890e-06,1.782890e-06,[1.68730163e+01 1.66658520e+01 1.69132795e+01 ...
4,ZTF17aaaizej,0.660175,0.404425,1.000000,1.000000,1.0,1.0,0.317073,0.311111,0.0,...,1.000000,0.009,2.871,4.185692,1.314692,4.359793,1.488793,4.163987e-08,6.982180e-05,[16.59538119 16.52297844 16.78657733 16.866158...


Obtener el tensor de cada evento

In [21]:
all_event_unique = df_features_tensor.oid.unique()
# event_unique
for index, event_unique in enumerate(all_event_unique):
    event = df_features_tensor.loc[df_features_tensor.oid == event_unique]
    print(event.Tensor)


0    [ 1.65146666e+01 -1.00000000e+03  1.64708068e+...
Name: Tensor, dtype: object
1    [1.54776381e+01 1.55176913e+01 1.56471603e+01 ...
Name: Tensor, dtype: object
2    [14.6432357  14.40605199 14.5821878  14.627640...
Name: Tensor, dtype: object
3    [1.68730163e+01 1.66658520e+01 1.69132795e+01 ...
Name: Tensor, dtype: object
4    [16.59538119 16.52297844 16.78657733 16.866158...
Name: Tensor, dtype: object
5    [ 1.55446462e+01  1.52905268e+01 -1.00000000e+...
Name: Tensor, dtype: object
6    [16.07101507 16.09323194 15.82592836 16.001691...
Name: Tensor, dtype: object
7    [13.31371896 13.36776297 13.25455894 13.328775...
Name: Tensor, dtype: object
8    [16.4159155  16.81547009 17.1519259  16.619421...
Name: Tensor, dtype: object
9    [16.57686554 16.26141619 16.52166201 16.598941...
Name: Tensor, dtype: object
10    [13.6907157  13.64819101 13.56432069 13.555009...
Name: Tensor, dtype: object
11    [ 1.70497592e+01 -1.00000000e+03  1.71117019e+...
Name: Tensor, dtype: object
12